In [ ]:
import sys
sys.path.append("..")
import reproducibility
import pandas as pd
from data_funcs import load_and_fix_data
from moisture_rnn import create_RNN_2, create_rnn_data_1, create_rnn_data_2, train_rnn

In [ ]:
reproducibility_file='../data/reproducibility_dict.pickle'
repro=load_and_fix_data(reproducibility_file)

In [ ]:
from module_param_sets import param_sets

In [ ]:
params = param_sets['0']

In [ ]:
# Simplify params
params['batch_size']=3
params["timesteps"]=2
params["epochs"]=1
params["initialize"]=False
params["hidden_units"]=1
params["rain_do"]=False
params

In [ ]:
# Format Data
case_data = repro["case11"]
h2=20
reproducibility.set_seed() # Set seed for reproducibility
rnn_dat = create_rnn_data_1(case_data,params)
create_rnn_data_2(rnn_dat,params)

In [ ]:
features = rnn_dat["X"].shape[1]
features

In [ ]:
# Setup Model
reproducibility.set_seed()
model = create_RNN_2(
    hidden_units=params["hidden_units"], 
    dense_units=1, 
    activation=params["activation"],
    batch_shape=(params["batch_size"],params["timesteps"],features),
    stateful=True
)
# Print initial weights
model.get_weights()

In [ ]:
# Run a sample through
X = rnn_dat["x_train"][0,:,:].reshape(-1,params["timesteps"],features)
X

In [ ]:
preds = model.predict(X)
preds[0]

In [ ]:
y = rnn_dat["y_train"][0].reshape(-1, 1)

In [ ]:
# Calculate MSE
(y - preds[0])**2

In [ ]:
# Use loss calculation from before to manually update weights
model.output

In [ ]:
model.optimizer.learning_rate.value

In [ ]:
reproducibility.set_seed()
history = model.fit(X, 
            y, 
            epochs=params["epochs"], 
            batch_size=params["batch_size"])

In [ ]:
history.history

In [ ]:
# Print Trained Weights
model.get_weights()